In [3]:
import pandas as pd
import re

# Eingabedatei mit Spalten: Title, Author
df_input = pd.read_excel("quellen.xlsx")

# Schlagworte zur NACE-Zuordnung
nace_schlagworte = {
    "A": ["Agriculture", "Food", "Palm Oil"],
    "B": ["Mining", "Oil", "Petroleum", "Raw Material"],
    "C13": ["Textiles", "Fashion", "Clothing"],
    "C20": ["Chemicals", "Pharmaceutical"],
    "C22": ["Plastics"],
    "C26": ["Electronics", "Technology"],
    "D": ["Utilities", "Waste", "Energy", "Power"],
    "G": ["Supply Chain", "Retail", "Trade"],
    "H": ["Transport"],
    "K": ["Financial", "Corporate Responsibility", "Investment"]
}

# Funktion zur Bewertung des Autors
def vertrauens_score(author: str) -> int:
    trusted_high = ["UN", "OECD", "ILO", "EU", "Bundesministerium", "University", "Max Planck"]
    trusted_medium = ["NGO", "Fairtrade", "McKinsey", "Oxfam", "Vision"]
    author = str(author)
    if any(key.lower() in author.lower() for key in trusted_high):
        return 10
    elif any(key.lower() in author.lower() for key in trusted_medium):
        return 7
    return 4

# Analyse durchführen
zuordnung = []

for _, row in df_input.iterrows():
    title = str(row["Title"])
    author = str(row["Author"])

    # NACE-Code zuordnen
    matched_codes = []
    for code, keywords in nace_schlagworte.items():
        if any(re.search(rf"\b{kw}\b", title, re.IGNORECASE) for kw in keywords):
            matched_codes.append(code)

    nace_codes = ", ".join(matched_codes) if matched_codes else "Unbestimmt"

    # Vertrauenswürdigkeit bewerten
    vertrauen = vertrauens_score(author)

    zuordnung.append({
        "Title": title,
        "Author": author,
        "NACE_Zuordnung": nace_codes,
        "Vertrauens_Score": vertrauen
    })

# DataFrame erstellen
df_output = pd.DataFrame(zuordnung)

# Ergebnis speichern
output_datei = "nace_quellenbewertung.xlsx"
df_output.to_excel(output_datei, index=False)

# Ausgabe
print(f"Verarbeitung abgeschlossen. Ergebnisse gespeichert in: {output_datei}")
print(df_output)


Verarbeitung abgeschlossen. Ergebnisse gespeichert in: nace_quellenbewertung.xlsx
                                                Title  \
0    Identifying the Human Rights Impacts of Palm Oil   
1   Handlungsleitfaden zur Durchführung der Risiko...   
2   UNEP FI Human Rights Guidance Tool for the Fin...   
3   Wages and Working Hours in the Textiles, Cloth...   
4             Praxislotse Wirtschaft & Menschenrechte   
5   UNEP FI Human Rights Guidance Tool for the Fin...   
6   UNEP FI Human Rights Guidance Tool for the Fin...   
7   Strengthening Protection Against Trafficking i...   
8      Human Rights Risks in Mining, A Baseline Study   
9   2022 List Of Goods Produced By Child Labor Or ...   
10  Die Achtung von Menschenrechten entlang global...   
11  Global Estimates of Modern Slavery, Forced Lab...   
12  Low Prices Drive Natural Rubber Producers Into...   
13                     Global Mercury Assessment 2018   
14  Sustainability and Circularity in the Textile ...   
15    

In [6]:
import pandas as pd
import re
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

# Schlagworte zur NACE-Zuordnung
nace_schlagworte = {
    "A": ["Agriculture", "Food", "Palm Oil"],
    "B": ["Mining", "Oil", "Petroleum", "Raw Material"],
    "C13": ["Textiles", "Fashion", "Clothing"],
    "C20": ["Chemicals", "Pharmaceutical"],
    "C22": ["Plastics"],
    "C26": ["Electronics", "Technology"],
    "D": ["Utilities", "Waste", "Energy", "Power"],
    "G": ["Supply Chain", "Retail", "Trade"],
    "H": ["Transport"],
    "K": ["Financial", "Corporate Responsibility", "Investment"]
}

def vertrauens_score(author: str) -> int:
    trusted_high = ["UN", "OECD", "ILO", "EU", "Bundesministerium", "University", "Max Planck"]
    trusted_medium = ["NGO", "Fairtrade", "McKinsey", "Oxfam", "Vision"]
    author = str(author)
    if any(key.lower() in author.lower() for key in trusted_high):
        return 10
    elif any(key.lower() in author.lower() for key in trusted_medium):
        return 7
    return 4

def analyse_datei(pfad):
    try:
        df_input = pd.read_excel(pfad)
    except Exception as e:
        messagebox.showerror("Fehler beim Laden", str(e))
        return

    zuordnung = []
    for _, row in df_input.iterrows():
        title = str(row["Title"])
        author = str(row["Author"])

        # NACE-Code zuordnen
        matched_codes = []
        for code, keywords in nace_schlagworte.items():
            if any(re.search(rf"\b{kw}\b", title, re.IGNORECASE) for kw in keywords):
                matched_codes.append(code)

        nace_codes = ", ".join(matched_codes) if matched_codes else "Unbestimmt"

        # Vertrauenswürdigkeit bewerten
        vertrauen = vertrauens_score(author)

        zuordnung.append({
            "Title": title,
            "Author": author,
            "NACE_Zuordnung": nace_codes,
            "Vertrauens_Score": vertrauen
        })

    # DataFrame erstellen
    df_output = pd.DataFrame(zuordnung)
    df_output.to_excel("nace_quellenbewertung.xlsx", index=False)
    return df_output

def lade_datei():
    pfad = filedialog.askopenfilename(filetypes=[("Excel-Dateien", "*.xlsx")])
    if not pfad:
        return
    df = analyse_datei(pfad)
    if df is not None:
        for row in tree.get_children():
            tree.delete(row)
        for _, row in df.iterrows():
            tree.insert("", tk.END, values=list(row))
        messagebox.showinfo("Fertig", "Analyse abgeschlossen und gespeichert als 'nace_quellenbewertung.xlsx'")

# UI erstellen
root = tk.Tk()
root.title("NACE Quellenanalyse Tool")
root.geometry("1000x600")
root.configure(bg="#f0f4f8")

frame_top = tk.Frame(root, bg="#f0f4f8")
frame_top.pack(pady=20)

lbl_title = tk.Label(frame_top, text="NACE-Zuordnung und Vertrauensbewertung", font=("Arial", 16, "bold"), bg="#f0f4f8")
lbl_title.pack(pady=5)

btn_laden = tk.Button(frame_top, text="Excel-Datei laden & analysieren", command=lade_datei, font=("Arial", 12), bg="#4CAF50", fg="white", padx=10, pady=5)
btn_laden.pack()

frame_table = tk.Frame(root)
frame_table.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

columns = ["Title", "Author", "NACE_Zuordnung", "Vertrauens_Score"]
tree = ttk.Treeview(frame_table, columns=columns, show="headings", height=20)

style = ttk.Style()
style.configure("Treeview.Heading", font=("Arial", 11, "bold"))
style.configure("Treeview", font=("Arial", 10))

for col in columns:
    tree.heading(col, text=col)
    tree.column(col, anchor="w", width=230)
tree.pack(fill=tk.BOTH, expand=True)

root.mainloop()
